# Validación de Condiciones Energéticas para D_{μν} (EDR)
Este notebook comprueba **NEC**, **WEC** y **SEC** para el tensor efectivo $D_{\mu\nu}$ en modelos *toy*.
Cuando tengas la expresión exacta de $D_{\mu\nu}$ (desde las derivaciones), reemplaza la función `D_mu_nu_toy` por la implementación real.

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
from numpy.linalg import eigvals

np.set_printoptions(precision=4, suppress=True)

## 1) Definiciones y tests (generales)
- **NEC:** $T_{\mu\nu}k^\mu k^\nu \ge 0$ para todo vector nulo $k^\mu$.
- **WEC:** $T_{\mu\nu}u^\mu u^\nu \ge 0$ para todo vector tiempo-likely $u^\mu$.
- **SEC:** $(T_{\mu\nu} - \tfrac12 T g_{\mu\nu})u^\mu u^\nu \ge 0$.
Aplicaremos estas condiciones a $D_{\mu\nu}$ (la interpretamos como contribución efectiva de energía-momento).

In [ ]:
def D_mu_nu_toy(r, theta, params):
    """Tensor simétrico toy D_{mu nu}(r,theta). Devuelve matriz 4x4.
    Dependencias: params dict con eta, xi, kflow, Omega0, R_Omega"""
    eta = params.get('eta', 0.02)
    xi = params.get('xi', 0.03)
    kflow = params.get('kflow', 0.01)
    Omega0 = params.get('Omega0', 0.1)
    # Construcción sencilla: diag(rho, p_r, p_t, p_t) in (-,+,+,+)
    rho = kflow * eta * (Omega0 * np.exp(-r/params.get('R_Omega',20)))**2
    p_r = 0.1 * rho
    p_t = 0.05 * rho
    # Matriz en coordenadas (t,r,theta,phi) con signatura (-,+,+,+)
    D = np.array([[-rho, 0, 0, 0],
                  [0, p_r, 0, 0],
                  [0, 0, p_t, 0],
                  [0, 0, 0, p_t]])
    return D

## 2) Vectores test: nulos y tiempo-like
Generamos vectores nulos $k^\mu$ y tiempo-like $u^\mu$ en un marco local (ortho-normal aproximado).

In [ ]:
def null_vector_example():
    # en un marco local simple, k=(1,1,0,0) es nulo si g=(-1,1,1,1)
    return np.array([1.0, 1.0, 0.0, 0.0])

def timelike_unit():
    return np.array([1.0, 0.0, 0.0, 0.0])

k = null_vector_example()
u = timelike_unit()
k, u

## 3) Evaluación de NEC, WEC, SEC en una malla (r,theta)
Se computa $D_{\mu\nu}k^\mu k^\nu$ y similares y se reportan regiones que violan o satisfacen cada condición.

In [ ]:
params = {'eta':0.02, 'xi':0.03, 'kflow':0.01, 'Omega0':0.1, 'R_Omega':20}
r_vals = np.logspace(np.log10(1.01), 2.7, 60)  # desde 1.01 hasta ~500
theta_vals = np.linspace(0, np.pi, 9)
results = {'NEC_violations': [], 'WEC_violations': [], 'SEC_violations': []}

for r in r_vals:
    for th in theta_vals:
        D = D_mu_nu_toy(r, th, params)
        k = null_vector_example()
        u = timelike_unit()
        NEC_val = float(k @ D @ k)
        WEC_val = float(u @ D @ u)
        Ttrace = np.trace(D)
        SEC_val = float((D - 0.5 * Ttrace * np.eye(4)) @ u @ u)
        if NEC_val < -1e-12:
            results['NEC_violations'].append((r, th, NEC_val))
        if WEC_val < -1e-12:
            results['WEC_violations'].append((r, th, WEC_val))
        if SEC_val < -1e-12:
            results['SEC_violations'].append((r, th, SEC_val))

results_summary = {k: len(v) for k, v in results.items()}
results_summary

## 4) Inspección de autovalores (condiciones más fuertes)
Analizamos autovalores de $D^\mu{}_{\nu}$ en el marco local: para WEC necesitamos que el componente proyectado con tiempos sea no negativo; analizar autovalores ayuda a ver presencia de modos fantasma.

In [ ]:
eigen_issues = []
for r in [1.1, 5, 20, 100]:
    D = D_mu_nu_toy(r, 0.3, params)
    # construir D^mu{}_nu usando g = diag(-1,1,1,1)
    g = np.diag([-1.0,1.0,1.0,1.0])
    D_up_down = g @ D
    eigs = np.linalg.eigvals(D_up_down)
    eigen_issues.append({'r': r, 'eigs': np.real_if_close(eigs)})

eigen_issues

## 5) Conclusiones provisionales
- Este notebook usa un tensor `toy` que satisface las condiciones para parámetros pequeños; al reemplazar con la expresión derivada índice-por-índice es donde se obtendrá la conclusión definitiva.
- Se generan salidas `results` y `eigen_issues` que puedes inspeccionar para crear figuras o tablas en el paper.

In [ ]:
with open('results/energy_conditions_summary.json','w') as f:
    json.dump({'summary': results_summary, 'eigen': eigen_issues}, f, indent=2)
print('Guardado results/energy_conditions_summary.json')